In [4]:
import pandas as pd
import matplotlib.pyplot as plt

def load_crop_data(file_path):
    """Load and prepare the crop data"""
    df = pd.read_csv(file_path)
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'June', 'July', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    return df, months

def predict_crops_with_scores(state, soil_type, month, df, months):
    """Predict suitable crops with confidence scores"""
    try:
        month_index = months.index(month)
    except ValueError:
        return "Invalid month. Please use one of: " + ", ".join(months)

    # Filter by state
    state_data = df[df['State'].str.lower() == state.lower()]
    if state_data.empty:
        return f"No data found for state: {state}"

    # Filter by soil type
    soil_data = state_data[state_data['Predominant Soil Types'].str.contains(soil_type, case=False)]
    if soil_data.empty:
        return f"No crops found for soil type: {soil_type} in state: {state}"

    # Filter by month availability (where value is 1)
    available_crops = soil_data[soil_data[month] == 1]

    if available_crops.empty:
        return f"No crops available in {month} for {state} with {soil_type} soil"

    # Calculate confidence scores based on various factors
    scores = []
    for _, crop_row in available_crops.iterrows():
        score = calculate_crop_score(crop_row, month, months)
        scores.append((crop_row['Commodity'], score))

    # Sort by score in descending order
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores

def calculate_crop_score(crop_row, current_month, months):
    """Calculate a confidence score for a crop based on various factors"""
    score = 0.0

    # Count number of growing months (more growing months = more reliable crop)
    growing_months = sum([1 for month in months if crop_row[month] == 1])
    score += growing_months / len(months) * 40  # Up to 40 points for growing season length

    # Check if it's in the middle of growing season
    month_index = months.index(current_month)
    prev_month = months[month_index - 1] if month_index > 0 else months[-1]
    next_month = months[(month_index + 1) % len(months)]

    if crop_row[prev_month] == 1 and crop_row[next_month] == 1:
        score += 30  # 30 points for being in middle of growing season

    # Add some base points for being a suitable crop
    score += 30

    return min(score, 100)  # Cap at 100

def classify_crops(crop_scores):
    vegetables = []
    main_crops = []
    fruits = []

    crop_classifications = {
        'Vegetable': ['Ashgourd', 'Bhindi(Ladies Finger)', 'Bitter gourd', 'Bottle gourd', 'Brinjal', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Cluster beans', 'Colacasia', 'Coriander(Leaves)', 'Cucumbar(Kheera)', 'Drumstick', 'Elephant Yam (Suran)', 'French Beans (Frasbean)', 'Green Chilli', 'Knool Khol', 'Leafy Vegetable', 'Little gourd (Kundru)', 'Onion', 'Potato', 'Pumpkin', 'Raddish', 'Ridgeguard(Tori)', 'Snakeguard', 'Spinach', 'Sweet Potato', 'Tapioca', 'Tomato','Beetroot','Beans','Sponge gourd','Squash(Chappal Kadoo)','Turnip','Rat Tail Radish (Mogari)','Kartali (Kantola)'],
        'Main Crop': ['Black Gram (Urd Beans)(Whole)', 'Cotton', 'Groundnut', 'Jowar(Sorghum)', 'Maize', 'Paddy(Dhan)(Common)', 'Ragi (Finger Millet)', 'Rice', 'Soyabean', 'Wheat','Arhar (Tur/Red Gram)(Whole)','Bajra(Pearl Millet/Cumbu)','Bengal Gram Dal (Chana Dal)','Bengal Gram(Gram)(Whole)','Castor Seed','Foxtail Millet(Navane)','Green Gram Dal (Moong Dal)','Ground Nut Oil','Groundnut','Gur(Jaggery)','Kulthi(Horse Gram)','Paddy(Dhan)(Basmati)','Rice','Sesamum(Sesame,Gingelly,Til)','Sunflower','Wheat Atta','Barley (Jau)','Kabuli Chana(Chickpeas-White)','Paddy(Dhan)(Basmati)','Red Gram','Alasande Gram','Ambady/Mesta','Guar Seed(Cluster Beans Seed)','Jute Seed','Kodo Millet(Varagu)','Lak(Teora)','Linseed','Millets','Niger Seed (Ramtil)','Rajgir','Safflower','Same/Savi','Sugarcane','Tobacco','White Peas','Groundnut pods (raw)','Guar','Mataki','Moath Dal','Raya','Cotton Seed','Indian Colza(Sarson)','Alsandikai','Avare Dal','Bunch Beans','Chapparad Avare','Chennangi Dal','Cowpea (Lobia/Karamani)','Duster Beans','Green Avare (W)','Ground Nut Seed','Honge seed','Sajje','Suvarna Gadde','Tamarind Seed','Thogrikai','Thondekai','White Pumpkin','Amphophalus','Coconut Oil','Coconut Seed','Rubber','Yam','Kutki','Pupadia','Binoula','Dhaincha','Groundnut (Split)','Hybrid Cumbu','Karamani','T.V. Cumbu','Thinai (Italian Millet)','Duck','Ambada Seed'],
        'Fruit': ['Apple', 'Banana', 'Grapes', 'Guava', 'Jack Fruit', 'Lemon', 'Lime', 'Mango', 'Orange', 'Papaya', 'Pineapple', 'Pomegranate', 'Water Melon','Karbuja(Musk Melon)','Mousambi(Sweet Lime)','Ber(Zizyphus/Borehannu)','Litchi','Plum','Pear(Marasebu)','Apricot(Jardalu/Khumani)','Cherry','Persimon(Japani Fal)','Custard Apple (Sharifa)','Dry Grapes','Chikoos(Sapota)','Amla(Nelli Kai)','Kinnow','Mango (Raw-Ripe)','Seetapal','Jamun(Narale Hannu)','Long Melon(Kakri)','Peach','Fig(Anjura/Anjeer)'],
    }

    if isinstance(crop_scores, str):
        return [], [], []

    for crop, probability in crop_scores:
        if crop in crop_classifications['Vegetable']:
            vegetables.append((crop, probability))
        elif crop in crop_classifications['Main Crop']:
            main_crops.append((crop, probability))
        elif crop in crop_classifications['Fruit']:
            fruits.append((crop, probability))
        else:
            main_crops.append((crop,probability)) #Default to main crop if not identified.
    return vegetables, main_crops, fruits

def display_classified_crops(vegetables, main_crops, fruits):
    print("\nVegetables:")
    print("------------------------")
    for i, (crop, score) in enumerate(vegetables, 1):
        print(f"{i:2d}. {crop:30s} ({score:.2f}%)")

    print("\nMain Crops:")
    print("------------------------")
    for i, (crop, score) in enumerate(main_crops, 1):
        print(f"{i:2d}. {crop:30s} ({score:.2f}%)")

    print("\nFruits:")
    print("------------------------")
    for i, (crop, score) in enumerate(fruits, 1):
        print(f"{i:2d}. {crop:30s} ({score:.2f}%)")

def visualize_crop_probabilities(crop_scores, top_n=20):
    """Visualize top N crop predictions"""
    if isinstance(crop_scores, str):
        print(crop_scores)
        return

    top_crops = crop_scores[:top_n]
    crops, scores = zip(*top_crops)

    plt.figure(figsize=(15, 8))
    bars = plt.bar(crops, scores)
    plt.title(f'Top {len(top_crops)} Crops Suitability Analysis')
    plt.xlabel('Crops')
    plt.ylabel('Suitability Score (%)')
    plt.xticks(rotation=45, ha='right')

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2., height, f'{height:.1f}%', ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

def main():
    try:
        # Load the data
        print("Loading crop data...")
        df, months = load_crop_data('crop.csv')

        print("\nCrop Prediction System")
        print("----------------------")

        # Get user input
        state = input("Enter state: ")
        soil_type = input("Enter soil type: ")
        print(f"Available months: {', '.join(months)}")
        month = input("Enter month: ")

        # Get predictions with scores
        results = predict_crops_with_scores(state, soil_type, month, df, months)

        # Classify crops
        vegetables, main_crops, fruits = classify_crops(results)

        # Display results
        print(f"\nAnalysis for:")
        print(f"State: {state}")
        print(f"Soil Type: {soil_type}")
        print(f"Month: {month}")

        # Display and visualize recommendations
        display_classified_crops(vegetables, main_crops, fruits)
        visualize_crop_probabilities(results)

    except FileNotFoundError:
        print("Error: Could not find the crop data file.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Loading crop data...

Crop Prediction System
----------------------
Available months: Jan, Feb, Mar, Apr, May, June, July, Aug, Sep, Oct, Nov, Dec

Analysis for:
State: Madhay Pradesh
Soil Type: Red Soil
Month: Jul

Vegetables:
------------------------

Main Crops:
------------------------

Fruits:
------------------------
Invalid month. Please use one of: Jan, Feb, Mar, Apr, May, June, July, Aug, Sep, Oct, Nov, Dec
